In [71]:
import torch
import torch.nn as nn
from torch import optim
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [72]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(34, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

model = Net()
print(model)

Net(
  (fc1): Linear(in_features=34, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
)


In [92]:
xs = []
ys = []

data_len = 10000

with open("txt/paddr_bin") as bookfile:
    i = 0
    for line in bookfile:
        str_list = list(line[:-1])
        int_list = list(map(int, str_list))
        tensor = torch.FloatTensor(int_list)
        xs.append(tensor)
        i += 1
        if (i == data_len): break

with open("txt/slice") as bookfile:
    i = 0
    for line in bookfile:
        str_list = list(line[:-1])
        int_list = list(map(int, str_list))
        tensor = torch.FloatTensor(int_list)
        ys.append(tensor)
        i += 1
        if (i == data_len): break

In [89]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [100]:
for epoch in range(10000):
    cost = 0.0

    for i in range(1000):
        x = xs[i]
        y = ys[i]

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(xs)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

KeyboardInterrupt: 

In [99]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(xs[9500])
    print(inputs)
    outputs = model(inputs)

print('--------------')
print(outputs)

tensor([0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
--------------
tensor([-0.0261])
